In [ ]:
%pip install -r requirements.txt

Basic Azure OpenAI Integration

In [ ]:
from openai import AzureOpenAI
import json
import random
import string

#Replace with your Azure OpenAI Key
aoai_key = "YOUR_AZURE_OPENAI_KEY"
aoai_api_version = "2024-02-15-preview"

#Replace with your Pay-i API Key
payi_api_key="YOUR_PAYI_API_KEY"

#Replace with your Azure OpenAI Model Name, e.g. "gpt-4o-2024-05-13"
#Note that the full model name is required by Azure OpenAI
aoai_model = "YOUR_AZURE_OPENAI_MODEL"

#Replace with your Azure OpenAI Deployment Name, e.g. "test-4o"
aoai_deployment = "YOUR_AZURE_OPENAI_DEPLOYMENT"

#Replace with your deployed Azure OpenAI endpoint URI
aoai_instance_uri = "YOUR_AZURE_OPENAI_ENDPOINT"

payi_headers = {
    "xProxy-Forward-api-key": aoai_key,
    "xProxy-Provider-BaseUri": aoai_instance_uri,
    "xProxy-RouteAs-Resource": aoai_model,
}

payi_base_url = "https://api.pay-i.com"
payi_aoai_url = payi_base_url + "/api/v1/proxy/azure.openai"

oai_client = AzureOpenAI(
    api_key=payi_api_key,
    api_version=aoai_api_version,
    azure_endpoint=payi_aoai_url,
    default_headers=payi_headers
)

response = oai_client.chat.completions.create(
    model=aoai_deployment,
    messages=[{"role": "user", "content": "Say 'this is a test'"}]
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

Handle streaming calls. xproxy_result is returned as part of the last chunk.

In [ ]:
stream = oai_client.chat.completions.create(
    model=aoai_deployment,
    messages=[{"role": "user", "content": "tell me a very short story"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
    if 'xproxy_result' in chunk.model_extra:
        print()
        print(json.dumps(chunk.model_extra['xproxy_result'], indent=4))

Use the Pay-i SDK to send a request with a request tag

In [ ]:
from payi.lib.helpers import create_headers

response = oai_client.chat.completions.create(
    model=aoai_deployment,
    messages=[{"role": "user", "content": "Say 'this is a request tag test'"}],
    extra_headers=create_headers(request_tags=["x", "y"])
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

Create a budget and make a request with that budget

In [ ]:
from payi import Payi

payi_client = Payi(
    payi_api_key=payi_api_key,
    base_url=payi_base_url
)

#Create a budget
budget_response = payi_client.budgets.create(
    #Use a random name for the budget
    budget_name=''.join(random.choices(string.ascii_letters + string.digits, k=6)), 
    max=12.50, #$12.50 USD
    budget_type="Liberal",
    budget_response_type="Allow",
    budget_tags=["example_budget"]
)

budget_name = budget_response.budget.budget_name
budget_id = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_name}")
print(f"Budget ID: {budget_id}")

#Make a request using the new budget and request tags
response = oai_client.chat.completions.create(
    model=aoai_deployment, 
    messages=[{"role": "user", "content": "Say 'this is a budget test'"}],
    extra_headers=create_headers(
        request_tags=["x", "y"], 
        budget_ids=[budget_id]
    )
)

completion = response.choices[0].message.content
print(completion)

xproxy_result = response.xproxy_result
print(json.dumps(xproxy_result, indent=4))

See budget status

In [ ]:
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.output.base}")

Make an ingest call with pre-computed token values

In [ ]:
response = payi_client.ingest.units(
    category="system.openai",
    resource="gpt-4o",
    input=50,
    output=100,
    budget_ids= [budget_id],
    request_tags = ["a", "b"]
)

print(f"Ingest request ID: {response.request_id}")
print(f"Input Base Cost: {response.cost.input.base}")
print(f"Output Base Cost: {response.cost.output.base}")
print(f"Total Base Cost: {response.cost.total.base}")

Reset a budget back to zero tracked cost

In [ ]:
response = payi_client.budgets.reset(budget_id=budget_id)
print(response.message)
print("State prior to reset: ")
print(f"Budget Name: {response.budget_history.budget_name}")
print(f"Budget ID: {response.budget_history.budget_id}")
print(f"Budget Tags: {response.budget_history.budget_tags}")
print(f"Budget Reset Timestamp: {response.budget_history.budget_reset_timestamp}")
print(f"Budget Input Base Cost: {response.budget_history.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget_history.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.total.base}")

Create a small, conservative budget that will prevent calls from happening that exceed the maximum, then capture the output.

In [ ]:
budget_response = payi_client.budgets.create(
    budget_name=''.join(random.choices(string.ascii_letters + string.digits, k=6)),  
    max=0.01, 
    budget_type="conservative",
    budget_response_type="block",
    budget_tags=["budget_block_example"]
)
cons_budget = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_response.budget.budget_name}")
print(f"Budget ID: {budget_response.budget.budget_id}")

try:
    response = oai_client.chat.completions.create(
        model=aoai_deployment,
        messages=[{"role": "user", "content": "tell me a short story about a toy"}],
        extra_headers=create_headers(
            request_tags=["toy_story"], 
            budget_ids=[cons_budget]
        )
    )
except Exception as e:
    print(json.dumps(e.body, indent=4))

List and then delete all budgets

In [ ]:
response = payi_client.budgets.list()
for budget in response.items:
    print("Deleting budget with id:" + budget.budget_id)
    payi_client.budgets.delete(budget.budget_id)